In [1]:
from sudoku_smt_solvers.benchmarks.benchmark_runner import BenchmarkRunner

In [2]:
runner = BenchmarkRunner(puzzles_dir='resources/benchmarks', results_dir='results', timeout=30)
runner.run_benchmarks()

Found 100 puzzle files
Running CVC5 on puzzle sudoku_25x25_Extremely Easy_20241212_223411_puzzle
Result: {'status': 'sat', 'solve_time': 0.8568863868713379, 'propagations': 1158}
Running DPLL on puzzle sudoku_25x25_Extremely Easy_20241212_223411_puzzle
Result: {'status': 'sat', 'solve_time': 0.48486828804016113, 'propagations': 190458}
Running DPLL(T) on puzzle sudoku_25x25_Extremely Easy_20241212_223411_puzzle
Result: {'status': 'sat', 'solve_time': 0.47147488594055176, 'propagations': 190458}
Running Z3 on puzzle sudoku_25x25_Extremely Easy_20241212_223411_puzzle
Result: {'status': 'sat', 'solve_time': 0.2613027095794678, 'propagations': 1783}
Running CVC5 on puzzle sudoku_25x25_Easy_20241212_223635_puzzle
Result: {'status': 'unsat', 'solve_time': 0.2296905517578125, 'propagations': 1267}
Running DPLL on puzzle sudoku_25x25_Easy_20241212_223635_puzzle
Result: {'status': 'unsat', 'solve_time': 0.23894071578979492, 'propagations': 190567}
Running DPLL(T) on puzzle sudoku_25x25_Easy_202